In [ ]:
#https://towardsdatascience.com/9-distance-measures-in-data-science-918109d069fa
#https://medium.com/@leobezerra_90682/python-r-messi-and-cristiano-d2b5278dbd5a
#https://towardsdatascience.com/clustering-in-detail-b439f31c56b7
#https://www.youtube.com/watch?v=Fqk3sdeAXMo
#https://www.youtube.com/watch?v=-XKCV3V26i4
#https://www.youtube.com/watch?v=D-SRERibbiU

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Tampermonkey autoscroll colab notebook
%%javascript

window.scroll_flag = true
window.scroll_exit = false
window.scroll_delay = 100

$(".output_scroll").each(function() {
    $(this)[0].scrollTop = $(this)[0].scrollHeight;
});

function callScrollToBottom() {
    setTimeout(scrollToBottom, window.scroll_delay);
}

function scrollToBottom() {
    if (window.scroll_exit) {
        return;
    }
    if (!window.scroll_flag) {
        callScrollToBottom();
        return;
    };
    
    $(".output_scroll").each(function() {
        if (!$(this).attr('scroll_checkbox')){
            window.scroll_flag = true;
            $(this).attr('scroll_checkbox',true);
            var div = document.createElement('div');
            var checkbox = document.createElement('input');
            checkbox.type = "checkbox";
            checkbox.onclick = function(){window.scroll_flag = checkbox.checked}
            checkbox.checked = "checked"
            div.append("Auto-Scroll-To-Bottom: ");
            div.append(checkbox);
            $(this).parent().before(div);
        }
        
        $(this)[0].scrollTop = $(this)[0].scrollHeight;
    });
    callScrollToBottom();
}
scrollToBottom();

<IPython.core.display.Javascript object>

In [ ]:
# CTRL + F10 HERE
#ou need to paste this script into the JavaScript console of the Colab browser window (Ctrl+Shift+i):
'''
function ClickConnect() {
  console.log('Working')
  document
    .querySelector('#top-toolbar > colab-connect-button')
    .shadowRoot.querySelector('#connect')
    .click()
}

setInterval(ClickConnect, 60000)
'''
#

"\nfunction ClickConnect() {\n  console.log('Working')\n  document\n    .querySelector('#top-toolbar > colab-connect-button')\n    .shadowRoot.querySelector('#connect')\n    .click()\n}\n\nsetInterval(ClickConnect, 60000)\n"

In [ ]:
'''#GPU count and name
!/usr/local/cuda/bin/nvcc --version
!nvidia-smi
#CPU
!lscpu |grep 'Model name'
#no.of sockets i.e available slots for physical processors
!lscpu | grep 'Socket(s):'
#no.of cores each processor is having 
!lscpu | grep 'Core(s) per socket:'
#no.of threads each core is having
!lscpu | grep 'Thread(s) per core'
#L3 cache
!lscpu | grep "L3 cache" 
#if it had turbo boost it would've shown Min and Max MHz also but it is only showing current frequency this means it always operates at shown frequency
!lscpu | grep "MHz"
#memory that we can use
!free -h --si | awk  '/Mem:/{print $2}'
#hard disk space that we can use
!df -h / | awk '{print $4}'
'''
#
!apt-get install inxi -y
!inxi -F

Reading package lists... Done
Building dependency tree       
Reading state information... Done
inxi is already the newest version (2.3.56-1).


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# boas práticas: criar uma célula com todas as bibliotecas necessárias
# importando as bibliotecas necessárias
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import numpy as np
import plotly as py
import plotly.graph_objs as go
from sklearn.cluster import KMeans
import warnings
import os
#warnings.filterwarnings("ignore")
py.offline.init_notebook_mode(connected = True)

In [ ]:
# http://dados.ufrn.br/dataset/turmas

path = '/content/drive/'+"MyDrive"+'/DataScience/Dados/';

# turmas dos 3 semestres
turmas_url_2018_02 = path+'turmas-2018.2.csv'
turmas_url_2019_02 = path+'turmas-2019.2.csv'
turmas_url_2018_01 = path+'turmas-2018.1.csv'
turmas_url_2019_01 = path+'turmas-2019.1.csv'
turmas_url_2020_01 = path+'turmas-2020.6.csv'

# matrículas em componentes dos 3 semestres
notas_url_2018_02 = path+'matricula-componente-20182.csv'
notas_url_2019_02 = path+'matricula-componente-20192.csv'
notas_url_2018_01 = path+'matricula-componente-20181.csv'
notas_url_2019_01 = path+'matricula-componente-20191.csv'
notas_url_2020_01 = path+'matricula-componente-20206.csv'

# componentes curriculares presenciais
disciplinas_url = path+'componentes-curriculares-presenciais.csv'

# docentes da ufrn
docentes_url = path+'docentes.csv'

turmas_2018_02 = pd.read_csv(turmas_url_2018_02, sep=';')
turmas_2019_02 = pd.read_csv(turmas_url_2019_02, sep=';')
turmas_2018_01 = pd.read_csv(turmas_url_2018_01, sep=';')
turmas_2019_01 = pd.read_csv(turmas_url_2019_01, sep=';')
turmas_2020_01 = pd.read_csv(turmas_url_2020_01, sep=';')

# matriculas em componentes em todas as disciplinas
notas_2018_02 = pd.read_csv(notas_url_2018_02, sep=';')
notas_2019_02 = pd.read_csv(notas_url_2019_02, sep=';')
notas_2018_01 = pd.read_csv(notas_url_2018_01, sep=';')
notas_2019_01 = pd.read_csv(notas_url_2019_01, sep=';')
notas_2020_01 = pd.read_csv(notas_url_2020_01, sep=';')

# failsafe dropna
'''
turmas_2018_01 = turmas_2018_01.dropna()
notas_2018_01 =  notas_2018_01.dropna()

turmas_2018_02 = turmas_2018_02.dropna()
notas_2018_02 =  notas_2018_02.dropna()

turmas_2019_01 = turmas_2019_01.dropna()
notas_2019_01 =  notas_2019_01.dropna()

turmas_2019_02 = turmas_2019_02.dropna()
notas_2019_02 =  notas_2019_02.dropna()

turmas_2020_01 = turmas_2020_01.dropna()
notas_2020_01 =  notas_2020_01.dropna()

'''

# componentes curriculares presenciais
# por alguma razão ou outra, esse dataset tinha algum lixo em linhas que ocasionava 
# erros de leitura, dessa forma foi realizado um filtro apenas por colunas mais importantes
disciplinas = pd.read_csv(disciplinas_url, 
                          sep=';', 
                          usecols=["id_componente","codigo","nivel","nome","unidade_responsavel"])

docentes = pd.read_csv(docentes_url, sep=';')

In [ ]:
# Adaptando o dataframe turma e disciplinas para o merge
# Esse procedimento é necessário pois o dataframe turma está codificado com ids

# A coluna id_componente precisa ser a mesma em ambos os dataframes
turmas_2018_02.rename(columns={'id_componente_curricular': 'id_componente'}, inplace=True)
turmas_2019_02.rename(columns={'id_componente_curricular': 'id_componente'}, inplace=True)
turmas_2018_01.rename(columns={'id_componente_curricular': 'id_componente'}, inplace=True)
turmas_2019_01.rename(columns={'id_componente_curricular': 'id_componente'}, inplace=True)
turmas_2020_01.rename(columns={'id_componente_curricular': 'id_componente'}, inplace=True)

# id_componente em ambos os dataframes precisam ser do mesmo tipo
turmas_2018_02.id_componente = turmas_2018_02.id_componente.astype(str)
turmas_2019_02.id_componente = turmas_2019_02.id_componente.astype(str)
turmas_2018_01.id_componente = turmas_2018_01.id_componente.astype(str)
turmas_2019_01.id_componente = turmas_2019_01.id_componente.astype(str)
turmas_2020_01.id_componente = turmas_2020_01.id_componente.astype(str)

disciplinas.id_componente = disciplinas.id_componente.astype(str)

# Limpando o dataframe disciplinas com linhas desnecessárias
disciplinas.dropna(inplace=True)
disciplinas = disciplinas[~disciplinas.id_componente.duplicated()]

In [ ]:
# Fazendo um merge entre o dataframe turmas e disciplinas utilizando como base a coluna id_componente
# https://pandas.pydata.org/pandas-docs/stable/merging.html
colunas = ["id_turma","siape","codigo","nivel_ensino","nome","unidade_responsavel",
          "situacao_turma","capacidade_aluno","descricao_horario"]

# merge os dois dataframes utilizando como base a coluna id_componente
merge_turmas_disciplinas_2018_02 = pd.merge(turmas_2018_02, disciplinas, on='id_componente')[colunas]
merge_turmas_disciplinas_2019_02 = pd.merge(turmas_2019_02, disciplinas, on='id_componente')[colunas]
merge_turmas_disciplinas_2018_01 = pd.merge(turmas_2018_01, disciplinas, on='id_componente')[colunas]
merge_turmas_disciplinas_2019_01 = pd.merge(turmas_2019_01, disciplinas, on='id_componente')[colunas]
merge_turmas_disciplinas_2020_01 = pd.merge(turmas_2020_01, disciplinas, on='id_componente')[colunas]

merge_turmas_disciplinas_2018_02.dropna(inplace=True)
merge_turmas_disciplinas_2019_02.dropna(inplace=True)
merge_turmas_disciplinas_2018_01.dropna(inplace=True)
merge_turmas_disciplinas_2019_01.dropna(inplace=True)
merge_turmas_disciplinas_2020_01.dropna(inplace=True)

In [ ]:
# filtrando as turmas com código IMD

turmas_imd_2018_02 = merge_turmas_disciplinas_2018_02; #merge_turmas_disciplinas_2018_02[merge_turmas_disciplinas_2018_02.codigo.str.contains("IMD")]
turmas_imd_2019_02 = merge_turmas_disciplinas_2019_02; #merge_turmas_disciplinas_2019_02[merge_turmas_disciplinas_2019_02.codigo.str.contains("IMD")]
turmas_imd_2018_01 = merge_turmas_disciplinas_2018_01; #merge_turmas_disciplinas_2018_02[merge_turmas_disciplinas_2018_02.codigo.str.contains("IMD")]
turmas_imd_2019_01 = merge_turmas_disciplinas_2019_01; #merge_turmas_disciplinas_2019_02[merge_turmas_disciplinas_2019_02.codigo.str.contains("IMD")]
turmas_imd_2020_01 = merge_turmas_disciplinas_2020_01; #merge_turmas_disciplinas_2020_01[merge_turmas_disciplinas_2020_01.codigo.str.contains("IMD")]

# utilizar apenas disciplinas com código "CONSOLIDADA"
turmas_imd_2018_02 = turmas_imd_2018_02[turmas_imd_2018_02.situacao_turma == 'CONSOLIDADA'] 
turmas_imd_2019_02 = turmas_imd_2019_02[turmas_imd_2019_02.situacao_turma == 'CONSOLIDADA']
turmas_imd_2018_01 = turmas_imd_2018_01[turmas_imd_2018_01.situacao_turma == 'CONSOLIDADA'] 
turmas_imd_2019_01 = turmas_imd_2019_01[turmas_imd_2019_01.situacao_turma == 'CONSOLIDADA']
turmas_imd_2020_01 = turmas_imd_2020_01[turmas_imd_2020_01.situacao_turma == 'CONSOLIDADA']

# Adaptar o nome da coluna id_docente_interno para siape
turmas_imd_2018_02.rename(columns={'siape': 'siape'}, inplace=True)
turmas_imd_2019_02.rename(columns={'siape': 'siape'}, inplace=True)
turmas_imd_2018_01.rename(columns={'siape': 'siape'}, inplace=True)
turmas_imd_2019_01.rename(columns={'siape': 'siape'}, inplace=True)
turmas_imd_2020_01.rename(columns={'siape': 'siape'}, inplace=True)

turmas_imd_2018_02.capacidade_aluno = turmas_imd_2018_02.capacidade_aluno.astype(np.int64)
turmas_imd_2019_02.capacidade_aluno = turmas_imd_2019_02.capacidade_aluno.astype(np.int64)
turmas_imd_2018_01.capacidade_aluno = turmas_imd_2018_01.capacidade_aluno.astype(np.int64)
turmas_imd_2019_01.capacidade_aluno = turmas_imd_2019_01.capacidade_aluno.astype(np.int64)
turmas_imd_2020_01.capacidade_aluno = turmas_imd_2020_01.capacidade_aluno.astype(np.int64)
'''
# Adaptar os tipos de dados em turmas_imd para o merge com o dataframe docentes
turmas_imd_2018_02.siape = turmas_imd_2018_02.siape.astype(np.int64)
turmas_imd_2019_02.siape = turmas_imd_2019_02.siape.astype(np.int64)
turmas_imd_2018_01.siape = turmas_imd_2018_01.siape.astype(np.int64)
turmas_imd_2019_01.siape = turmas_imd_2019_01.siape.astype(np.int64)
turmas_imd_2020_01.siape = turmas_imd_2020_01.siape.astype(np.int64)
'''
#

In [ ]:
# merge os dataframes  turmas_imd e docentes utilizando como base a coluna id_servidor
colunas = ["id_turma", "nome_y", "codigo", "nivel_ensino",
          "nome_x","capacidade_aluno", "descricao_horario"]
turmas_imd_2018_02_nomesprof = pd.merge(turmas_imd_2018_02, docentes, on='siape')[colunas]
turmas_imd_2019_02_nomesprof = pd.merge(turmas_imd_2019_02, docentes, on='siape')[colunas]
turmas_imd_2018_01_nomesprof = pd.merge(turmas_imd_2018_01, docentes, on='siape')[colunas]
turmas_imd_2019_01_nomesprof = pd.merge(turmas_imd_2019_01, docentes, on='siape')[colunas]
turmas_imd_2020_01_nomesprof = pd.merge(turmas_imd_2020_01, docentes, on='siape')[colunas]

In [ ]:
# This is a checkpoint CTRL + F10 HERE

In [ ]:
notas_imd_2018_02 = notas_2018_02[notas_2018_02.id_turma.isin(turmas_imd_2018_02_nomesprof.id_turma)]
notas_imd_2019_02 = notas_2019_02[notas_2019_02.id_turma.isin(turmas_imd_2019_02_nomesprof.id_turma)]
notas_imd_2018_01 = notas_2018_01[notas_2018_01.id_turma.isin(turmas_imd_2018_01_nomesprof.id_turma)]
notas_imd_2019_01 = notas_2019_01[notas_2019_01.id_turma.isin(turmas_imd_2019_01_nomesprof.id_turma)]
notas_imd_2020_01 = notas_2020_01[notas_2020_01.id_turma.isin(turmas_imd_2020_01_nomesprof.id_turma)]

# merge entre os dataframes notas_imd e turmas_imd_nomesprof utilizando como referencia id_turma
notas_imd_2018_02_merge = pd.merge(notas_imd_2018_02, turmas_imd_2018_02_nomesprof, on='id_turma')
notas_imd_2019_02_merge = pd.merge(notas_imd_2019_02, turmas_imd_2019_02_nomesprof, on='id_turma')
notas_imd_2018_01_merge = pd.merge(notas_imd_2018_01, turmas_imd_2018_01_nomesprof, on='id_turma')
notas_imd_2019_01_merge = pd.merge(notas_imd_2019_01, turmas_imd_2019_01_nomesprof, on='id_turma')
notas_imd_2020_01_merge = pd.merge(notas_imd_2020_01, turmas_imd_2020_01_nomesprof, on='id_turma')


In [ ]:
notas_imd_2018_01_merge = notas_imd_2018_01_merge.replace({',': '.'}, regex=True)
notas_imd_2018_01_merge.nota = notas_imd_2018_01_merge.nota.astype(np.float64)
notas_imd_2018_01_merge.media_final = notas_imd_2018_01_merge.media_final.astype(np.float64)
notas_imd_2018_01_merge.id_curso = notas_imd_2018_01_merge.id_curso.fillna(0).astype(np.ushort)
notas_imd_2018_01_merge.unidade = notas_imd_2018_01_merge.unidade.fillna(0).astype(np.ushort)
notas_imd_2018_01_merge.faltas_unidade = notas_imd_2018_01_merge.faltas_unidade.fillna(0).astype(np.ushort)
notas_imd_2018_01_merge.numero_total_faltas = notas_imd_2018_01_merge.numero_total_faltas.fillna(0).astype(np.ushort)
notas_imd_2018_01_merge.capacidade_aluno = notas_imd_2018_01_merge.capacidade_aluno.fillna(0).astype(np.ushort)
notas_imd_2018_01_merge.id_turma = notas_imd_2018_01_merge.id_turma.fillna(0).astype(np.ushort)

notas_imd_2018_02_merge = notas_imd_2018_02_merge.replace({',': '.'}, regex=True)
notas_imd_2018_02_merge.nota = notas_imd_2018_02_merge.nota.astype(np.float64)
notas_imd_2018_02_merge.media_final = notas_imd_2018_02_merge.media_final.astype(np.float64)
notas_imd_2018_02_merge.id_curso = notas_imd_2018_02_merge.id_curso.fillna(0).astype(np.ushort)
notas_imd_2018_02_merge.unidade = notas_imd_2018_02_merge.unidade.fillna(0).astype(np.ushort)
notas_imd_2018_02_merge.faltas_unidade = notas_imd_2018_02_merge.faltas_unidade.fillna(0).astype(np.ushort)
notas_imd_2018_02_merge.numero_total_faltas = notas_imd_2018_02_merge.numero_total_faltas.fillna(0).astype(np.ushort)
notas_imd_2018_02_merge.capacidade_aluno = notas_imd_2018_02_merge.capacidade_aluno.fillna(0).astype(np.ushort)
notas_imd_2018_02_merge.id_turma = notas_imd_2018_02_merge.id_turma.fillna(0).astype(np.ushort)

notas_imd_2019_01_merge = notas_imd_2019_01_merge.replace({',': '.'}, regex=True)
notas_imd_2019_01_merge.nota = notas_imd_2019_01_merge.nota.astype(np.float64)
notas_imd_2019_01_merge.media_final = notas_imd_2019_01_merge.media_final.astype(np.float64)
notas_imd_2019_01_merge.id_curso = notas_imd_2019_01_merge.id_curso.fillna(0).astype(np.ushort)
notas_imd_2019_01_merge.unidade = notas_imd_2019_01_merge.unidade.fillna(0).astype(np.ushort)
notas_imd_2019_01_merge.faltas_unidade = notas_imd_2019_01_merge.faltas_unidade.fillna(0).astype(np.ushort)
notas_imd_2019_01_merge.numero_total_faltas = notas_imd_2019_01_merge.numero_total_faltas.fillna(0).astype(np.ushort)
notas_imd_2019_01_merge.capacidade_aluno = notas_imd_2019_01_merge.capacidade_aluno.fillna(0).astype(np.ushort)
notas_imd_2019_01_merge.id_turma = notas_imd_2019_01_merge.id_turma.fillna(0).astype(np.ushort)

notas_imd_2019_02_merge = notas_imd_2019_02_merge.replace({',': '.'}, regex=True)
notas_imd_2019_02_merge.nota = notas_imd_2019_02_merge.nota.astype(np.float64)
notas_imd_2019_02_merge.media_final = notas_imd_2019_02_merge.media_final.astype(np.float64)
notas_imd_2019_02_merge.id_curso = notas_imd_2019_02_merge.id_curso.fillna(0).astype(np.ushort)
notas_imd_2019_02_merge.unidade = notas_imd_2019_02_merge.unidade.fillna(0).astype(np.ushort)
notas_imd_2019_02_merge.faltas_unidade = notas_imd_2019_02_merge.faltas_unidade.fillna(0).astype(np.ushort)
notas_imd_2019_02_merge.numero_total_faltas = notas_imd_2019_02_merge.numero_total_faltas.fillna(0).astype(np.ushort)
notas_imd_2019_02_merge.capacidade_aluno = notas_imd_2019_02_merge.capacidade_aluno.fillna(0).astype(np.ushort)
notas_imd_2019_02_merge.id_turma = notas_imd_2019_02_merge.id_turma.fillna(0).astype(np.ushort)

notas_imd_2020_01_merge = notas_imd_2020_01_merge.replace({',': '.'}, regex=True)
notas_imd_2020_01_merge.nota = notas_imd_2020_01_merge.nota.astype(np.float64)
notas_imd_2020_01_merge.media_final = notas_imd_2020_01_merge.media_final.astype(np.float64)
notas_imd_2020_01_merge.id_curso = notas_imd_2020_01_merge.id_curso.fillna(0).astype(np.ushort)
notas_imd_2020_01_merge.unidade = notas_imd_2020_01_merge.unidade.fillna(0).astype(np.ushort)
notas_imd_2020_01_merge.faltas_unidade = notas_imd_2020_01_merge.faltas_unidade.fillna(0).astype(np.ushort)
notas_imd_2020_01_merge.numero_total_faltas = notas_imd_2020_01_merge.numero_total_faltas.fillna(0).astype(np.ushort)
notas_imd_2020_01_merge.capacidade_aluno = notas_imd_2020_01_merge.capacidade_aluno.fillna(0).astype(np.ushort)
notas_imd_2020_01_merge.id_turma = notas_imd_2020_01_merge.id_turma.fillna(0).astype(np.ushort)


In [ ]:
def aprovado(row):
    if row["descricao"] == "APROVADO" or row["descricao"] == "APROVADO POR NOTA":
        return True
    else:
        return False

In [ ]:
def reprovadoporfalta(row):
    if row["descricao"] == "REPROVADO POR FALTAS" or row["descricao"] == "REPROVADO POR NOTA E FALTA" or row["descricao"] == "REPROVADO POR MÉDIA E POR FALTAS":
        return True
    else:
        return False

In [ ]:
def reposicao(row):
    if row["reposicao"] == "True":
        return True
    else:
        return False

In [ ]:
def matematicamentereprovado(row):
  if row["aprovado"] == 0:
    if row["media_final"] < 5:
      if row["reprovadoporfalta"] == True:
        return True;
        # ele tem que sair
      else:
        return False;
    else:
      return False;
      # ele tem que sair
  else:
    return False;

In [ ]:
# Colocando uma flag nos alunos aprovados
rotulos_2018_02 = notas_imd_2018_02_merge.apply(aprovado, axis = 1)
notas_imd_2018_02_merge["aprovado"] = rotulos_2018_02
rotulos_2019_02 = notas_imd_2019_02_merge.apply(aprovado, axis = 1)
notas_imd_2019_02_merge["aprovado"] = rotulos_2019_02
rotulos_2018_01 = notas_imd_2018_01_merge.apply(aprovado, axis = 1)
notas_imd_2018_01_merge["aprovado"] = rotulos_2018_01
rotulos_2019_01 = notas_imd_2019_01_merge.apply(aprovado, axis = 1)
notas_imd_2019_01_merge["aprovado"] = rotulos_2019_01
rotulos_2020_01 = notas_imd_2020_01_merge.apply(aprovado, axis = 1)
notas_imd_2020_01_merge["aprovado"] = rotulos_2020_01

In [ ]:
# Colocando uma flag nos alunos reprovados por falta
rotulos_2018_02 = notas_imd_2018_02_merge.apply(reprovadoporfalta, axis = 1)
notas_imd_2018_02_merge["reprovadoporfalta"] = rotulos_2018_02
rotulos_2019_02 = notas_imd_2019_02_merge.apply(reprovadoporfalta, axis = 1)
notas_imd_2019_02_merge["reprovadoporfalta"] = rotulos_2019_02
rotulos_2018_01 = notas_imd_2018_01_merge.apply(reprovadoporfalta, axis = 1)
notas_imd_2018_01_merge["reprovadoporfalta"] = rotulos_2018_01
rotulos_2019_01 = notas_imd_2019_01_merge.apply(reprovadoporfalta, axis = 1)
notas_imd_2019_01_merge["reprovadoporfalta"] = rotulos_2019_01
rotulos_2020_01 = notas_imd_2020_01_merge.apply(reprovadoporfalta, axis = 1)
notas_imd_2020_01_merge["reprovadoporfalta"] = rotulos_2020_01

In [ ]:
# Corrigindo o tipo da coluna reposicao
rotulos_2018_02 = notas_imd_2018_02_merge.apply(reposicao, axis = 1)
notas_imd_2018_02_merge["reposicao"] = rotulos_2018_02
rotulos_2019_02 = notas_imd_2019_02_merge.apply(reposicao, axis = 1)
notas_imd_2019_02_merge["reposicao"] = rotulos_2019_02
rotulos_2018_01 = notas_imd_2018_01_merge.apply(reposicao, axis = 1)
notas_imd_2018_01_merge["reposicao"] = rotulos_2018_01
rotulos_2019_01 = notas_imd_2019_01_merge.apply(reposicao, axis = 1)
notas_imd_2019_01_merge["reposicao"] = rotulos_2019_01
rotulos_2020_01 = notas_imd_2020_01_merge.apply(reposicao, axis = 1)
notas_imd_2020_01_merge["reposicao"] = rotulos_2020_01

In [ ]:
# Checkpoint CTRL + F10 HERE

In [ ]:
notas_imd_2018_02_merge['semestre'] = '2018.2'
notas_imd_2019_02_merge['semestre'] = '2019.2'
notas_imd_2018_01_merge['semestre'] = '2018.1'
notas_imd_2019_01_merge['semestre'] = '2019.1'
notas_imd_2020_01_merge['semestre'] = '2020.1'

df = pd.concat([notas_imd_2018_01_merge, notas_imd_2018_02_merge, notas_imd_2019_01_merge, notas_imd_2019_02_merge, notas_imd_2020_01_merge], ignore_index=True)
df.info()

In [ ]:
df.size

In [ ]:
df = df.dropna()
df = df.drop_duplicates()

In [ ]:
df.reset_index(drop=True, inplace=True)
df = df.drop_duplicates()

In [ ]:
df.size

In [ ]:
df.head()

In [ ]:
aux = df
df = aux.rename(columns = {'nome_y' : 'docente', 'nome_x' : 'disciplina'})

In [ ]:
dfc = df[['discente', 'unidade', 'nota', 'media_final','aprovado', 'reposicao', 'reprovadoporfalta', 'numero_total_faltas', 'descricao','nivel_ensino', 'id_turma', 'codigo', 'disciplina', 'docente','semestre']]
dfc.head()

In [ ]:
dfc.size

In [ ]:
#dfc = dfc[dfc.duplicated(subset = ["discente", "semestre"], keep = 'first') != True]

In [ ]:
dfc.descricao.unique()

In [ ]:
dfc.size

In [ ]:
# Fim de tratamento e limpeza, início de recorte e seleção

In [ ]:
dfc = dfc[dfc["descricao"] != "INDEFERIDO"]
dfc = dfc[dfc["descricao"] != "EXCLUIDA"]
dfc = dfc[dfc["descricao"] != "TRANCADO"]
dfc = dfc[dfc["descricao"] != "CANCELADO"]
dfc = dfc[dfc["descricao"] != "DESISTENCIA"]
dfc = dfc[dfc["descricao"] != "AGUARDANDO DEFERIMENTO"]
dfc = dfc[dfc["descricao"] != "CUMPRIU"]
dfc = dfc[dfc["descricao"] != "EM ESPERA"]
dfc = dfc[dfc["descricao"] != "DISPENSADO"]

In [ ]:
dfc.descricao.unique()

In [ ]:
dfc = dfc[dfc["nivel_ensino"] == "GRADUAÇÃO"]

In [ ]:
dfc.to_csv("dfc.csv")

In [ ]:
pensamento_computacional = df
pensamento_computacional = pensamento_computacional.loc[pensamento_computacional['disciplina'] == 'PENSAMENTO COMPUTACIONAL']
pensamento_computacional = pensamento_computacional.loc[pensamento_computacional['semestre'] == '2020.1']
pensamento_computacional = pensamento_computacional[['discente', 'unidade', 'nota', 'media_final','descricao', 'numero_total_faltas', 'disciplina','semestre']]
pensamento_computacional = pensamento_computacional.drop_duplicates()
pensamento_computacional.reset_index(drop=True, inplace=True)
pensamento_computacional = pensamento_computacional.drop_duplicates()
pensamento_computacional.head()

In [ ]:
pensamento_computacional.to_csv("pensamento_computacional.csv")

In [ ]:
pensamento_computacional.size

In [ ]:
dfc.size

In [ ]:
filtrados = dfc
filtrados = filtrados.loc[filtrados['disciplina'] == 'INTRODUÇÃO ÀS TÉCNICAS DE PROGRAMAÇÃO']
'''
rotulos_filtrados_2 = filtrados.apply(aprovado, axis = 1)
filtrados["aprovado"] = rotulos_filtrados_2
'''
rotulos_filtrados = filtrados.apply(matematicamentereprovado, axis = 1)
filtrados["reprovadomatematicamente"] = rotulos_filtrados

base_final = filtrados.loc[filtrados['reprovadomatematicamente'] == False]

In [ ]:
excluidos = filtrados.loc[filtrados['reprovadomatematicamente'] == True]

In [ ]:
excluidos.head()

In [ ]:
excluidos.describe()

In [ ]:
# linhas excluidas pela regra
excluidos.size

In [ ]:
# discentes excluidos pela regra
excluidos[excluidos.duplicated(subset = ["discente", "semestre"], keep = 'first') != True].size

In [ ]:
base_final.head()

In [ ]:
base_final.describe()

In [ ]:
base_final.info()

In [ ]:
base_final.to_csv("base_final.csv")

In [ ]:
# discentes na base final
base_final[base_final.duplicated(subset = ["discente", "semestre"], keep = 'first') != True].size

In [ ]:
base_final.size

In [ ]:
#Separando e tratando os alunos de graduação
graduacao_imd_2018_02 = notas_imd_2018_02_merge[notas_imd_2018_02_merge["nivel_ensino"] == "GRADUAÇÃO"]
graduacao_imd_2018_01 = notas_imd_2018_01_merge[notas_imd_2018_01_merge["nivel_ensino"] == "GRADUAÇÃO"]
graduacao_imd_2020_01 = notas_imd_2020_01_merge[notas_imd_2020_01_merge["nivel_ensino"] == "GRADUAÇÃO"]
graduacao_imd_2019_02 = notas_imd_2019_02_merge[notas_imd_2019_02_merge["nivel_ensino"] == "GRADUAÇÃO"]
graduacao_imd_2019_01 = notas_imd_2019_01_merge[notas_imd_2019_01_merge["nivel_ensino"] == "GRADUAÇÃO"]

In [ ]:
#Separando as turmas da disciplina de itp em cada semestre

graduacao_imd_2018_02 = graduacao_imd_2018_02.rename(columns = {'nome_y' : 'docente', 'nome_x' : 'disciplina'})
#graduacao_imd_2018_02.sort_values(ascending = True, by = ["id_turma", "discente","unidade"], inplace = True)
graduacao_imd_2018_02.sort_values(ascending = True, by = "discente", inplace = True)
graduacao_imd_2018_02.reset_index(inplace = True)
materias_2018_02 = graduacao_imd_2018_02.disciplina.unique()

graduacao_imd_2018_01 = graduacao_imd_2018_01.rename(columns = {'nome_y' : 'docente', 'nome_x' : 'disciplina'})
#graduacao_imd_2018_01.sort_values(ascending = True, by = ["id_turma", "discente","unidade"], inplace = True)
graduacao_imd_2018_01.sort_values(ascending = True, by = "discente", inplace = True)
graduacao_imd_2018_01.reset_index(inplace = True)
materias_2018_01 = graduacao_imd_2018_01.disciplina.unique()

graduacao_imd_2020_01 = graduacao_imd_2020_01.rename(columns = {'nome_y' : 'docente', 'nome_x' : 'disciplina'})
#graduacao_imd_2020_01.sort_values(ascending = True, by = ["id_turma", "discente","unidade"], inplace = True)
graduacao_imd_2020_01.sort_values(ascending = True, by = "discente", inplace = True)
graduacao_imd_2020_01.reset_index(inplace = True)
materias_2020_01 = graduacao_imd_2020_01.disciplina.unique()

graduacao_imd_2019_02 = graduacao_imd_2019_02.rename(columns = {'nome_y' : 'docente', 'nome_x' : 'disciplina'})
#graduacao_imd_2019_02.sort_values(ascending = True, by = ["id_turma", "discente","unidade"], inplace = True)
graduacao_imd_2019_02.sort_values(ascending = True, by = "discente", inplace = True)
graduacao_imd_2019_02.reset_index(inplace = True)
materias_2019_02 = graduacao_imd_2019_02.disciplina.unique()

graduacao_imd_2019_01 = graduacao_imd_2019_01.rename(columns = {'nome_y' : 'docente', 'nome_x' : 'disciplina'})
#graduacao_imd_2019_01.sort_values(ascending = True, by = ["id_turma", "discente","unidade"], inplace = True)
graduacao_imd_2019_01.sort_values(ascending = True, by = "discente", inplace = True)
graduacao_imd_2019_01.reset_index(inplace = True)
materias_2019_01 = graduacao_imd_2019_01.disciplina.unique()

graduacao_imd_2018_02 = graduacao_imd_2018_02.loc[graduacao_imd_2018_02['disciplina'] == 'INTRODUÇÃO ÀS TÉCNICAS DE PROGRAMAÇÃO']
graduacao_imd_2018_01 = graduacao_imd_2018_01.loc[graduacao_imd_2018_01['disciplina'] == 'INTRODUÇÃO ÀS TÉCNICAS DE PROGRAMAÇÃO']
graduacao_imd_2020_01 = graduacao_imd_2020_01.loc[graduacao_imd_2020_01['disciplina'] == 'INTRODUÇÃO ÀS TÉCNICAS DE PROGRAMAÇÃO']
graduacao_imd_2019_02 = graduacao_imd_2019_02.loc[graduacao_imd_2019_02['disciplina'] == 'INTRODUÇÃO ÀS TÉCNICAS DE PROGRAMAÇÃO']
graduacao_imd_2019_01 = graduacao_imd_2019_01.loc[graduacao_imd_2019_01['disciplina'] == 'INTRODUÇÃO ÀS TÉCNICAS DE PROGRAMAÇÃO']

graduacao_imd_2018_01 = graduacao_imd_2018_01.drop_duplicates()
graduacao_imd_2018_01.reset_index(drop=True, inplace=True)
graduacao_imd_2018_01 = graduacao_imd_2018_01.drop_duplicates()

graduacao_imd_2018_02 = graduacao_imd_2018_02.drop_duplicates()
graduacao_imd_2018_02.reset_index(drop=True, inplace=True)
graduacao_imd_2018_02 = graduacao_imd_2018_02.drop_duplicates()

graduacao_imd_2019_01 = graduacao_imd_2019_01.drop_duplicates()
graduacao_imd_2019_01.reset_index(drop=True, inplace=True)
graduacao_imd_2019_01 = graduacao_imd_2019_01.drop_duplicates()

graduacao_imd_2019_02 = graduacao_imd_2019_02.drop_duplicates()
graduacao_imd_2019_02.reset_index(drop=True, inplace=True)
graduacao_imd_2019_02 = graduacao_imd_2019_02.drop_duplicates()

graduacao_imd_2020_01 = graduacao_imd_2020_01.drop_duplicates()
graduacao_imd_2020_01.reset_index(drop=True, inplace=True)
graduacao_imd_2020_01 = graduacao_imd_2020_01.drop_duplicates()

In [ ]:
print('2018.1:', end =" ")
for codigo in graduacao_imd_2018_01.codigo.unique():
  print(codigo, end =" ")
print('\n')
print('2018.2:', end =" ")
for codigo in graduacao_imd_2018_02.codigo.unique():
  print(codigo, end =" ")
print('\n')
print('2019.1:', end =" ")
for codigo in graduacao_imd_2019_01.codigo.unique():
  print(codigo, end =" ")
print('\n')
print('2019.2:', end =" ")
for codigo in graduacao_imd_2019_02.codigo.unique():
  print(codigo, end =" ")
print('\n')
print('2020.1:', end =" ")
for codigo in graduacao_imd_2020_01.codigo.unique():
  print(codigo, end =" ")
print('\n')

In [ ]:
graduacao_imd_2018_02 = graduacao_imd_2018_02.replace({',': '.'}, regex=True)
graduacao_imd_2018_01 = graduacao_imd_2018_01.replace({',': '.'}, regex=True)
graduacao_imd_2020_01 = graduacao_imd_2020_01.replace({',': '.'}, regex=True)
graduacao_imd_2019_02 = graduacao_imd_2019_02.replace({',': '.'}, regex=True)
graduacao_imd_2019_01 = graduacao_imd_2019_01.replace({',': '.'}, regex=True)

graduacao_imd_2018_02.nota = graduacao_imd_2018_02.nota.astype(np.float64)
graduacao_imd_2018_01.nota = graduacao_imd_2018_01.nota.astype(np.float64)
graduacao_imd_2020_01.nota = graduacao_imd_2020_01.nota.astype(np.float64)
graduacao_imd_2019_02.nota = graduacao_imd_2019_02.nota.astype(np.float64)
graduacao_imd_2019_01.nota = graduacao_imd_2019_01.nota.astype(np.float64)

graduacao_imd_2018_02.media_final = graduacao_imd_2018_02.media_final.astype(np.float64)
graduacao_imd_2018_01.media_final = graduacao_imd_2018_01.media_final.astype(np.float64)
graduacao_imd_2020_01.media_final = graduacao_imd_2020_01.media_final.astype(np.float64)
graduacao_imd_2019_02.media_final = graduacao_imd_2019_02.media_final.astype(np.float64)
graduacao_imd_2019_01.media_final = graduacao_imd_2019_01.media_final.astype(np.float64)



#COMMENT

In [ ]:
fig = plt.figure(figsize=(50, 100));
labels = ['APROVADO' 'REPROVADO' 'APROVADO POR NOTA' 'REPROVADO POR NOTA'
'DESISTENCIA' 'REPROVADO POR MÉDIA E POR FALTAS'
'REPROVADO POR NOTA E FALTA']


columns_labels = {
    'APROVADO': 'green',
    'REPROVADO': 'red',
    'APROVADO POR NOTA': 'darkgreen',
    'REPROVADO POR NOTA': 'tomato',
    'REPROVADO POR FALTAS': 'crimson',
    'DESISTENCIA': 'silver',
    'REPROVADO POR MÉDIA E POR FALTAS': 'coral',
    'REPROVADO POR NOTA E FALTA': 'maroon',
    'TRANCADO': 'gray',
}


axx1 = plt.subplot(1, 5, 1);
pxxx1 = graduacao_imd_2018_01[graduacao_imd_2018_01.duplicated(subset = ["discente", "semestre"], keep = 'first') != True]
pxxx1 = pxxx1[pxxx1["descricao"] != "INDEFERIDO"]
pxxx1 = pxxx1[pxxx1["descricao"] != "EXCLUIDA"]
pxxx1 = pxxx1[pxxx1["descricao"] != "CANCELADO"]

pxxx1 = pxxx1.descricao.value_counts()
plt.title('2018.1');
pxxx1.plot(kind='pie', autopct='%1.1f%%', label='', colors=[columns_labels[v] for v in pxxx1.keys()])


axx4 = plt.subplot(1, 5, 2);
pxxx4 = graduacao_imd_2018_02[graduacao_imd_2018_02.duplicated(subset = ["discente", "semestre"], keep = 'first') != True]
pxxx4 = pxxx4[pxxx4["descricao"] != "INDEFERIDO"]
pxxx4 = pxxx4[pxxx4["descricao"] != "EXCLUIDA"]
pxxx4 = pxxx4[pxxx4["descricao"] != "CANCELADO"]

pxxx4 = pxxx4.descricao.value_counts()
plt.title('2018.2');
pxxx4.plot(kind='pie', autopct='%1.1f%%', label='', colors=[columns_labels[v] for v in pxxx4.keys()])


axx2 = plt.subplot(1, 5, 3);
pxxx2 = graduacao_imd_2019_01[graduacao_imd_2019_01.duplicated(subset = ["discente", "semestre"], keep = 'first') != True]
pxxx2 = pxxx2[pxxx2["descricao"] != "INDEFERIDO"]
pxxx2 = pxxx2[pxxx2["descricao"] != "EXCLUIDA"]
pxxx2 = pxxx2[pxxx2["descricao"] != "CANCELADO"]

pxxx2 = pxxx2.descricao.value_counts()
plt.title('2019.1');
pxxx2.plot(kind='pie', autopct='%1.1f%%', label='', colors=[columns_labels[v] for v in pxxx2.keys()])


axx5 = plt.subplot(1, 5, 4);
pxxx5 = graduacao_imd_2019_02[graduacao_imd_2019_02.duplicated(subset = ["discente", "semestre"], keep = 'first') != True]
pxxx5 = pxxx5[pxxx5["descricao"] != "INDEFERIDO"]
pxxx5 = pxxx5[pxxx5["descricao"] != "EXCLUIDA"]
pxxx5 = pxxx5[pxxx5["descricao"] != "CANCELADO"]

pxxx5 = pxxx5.descricao.value_counts()
plt.title('2019.2');
pxxx5.plot(kind='pie', autopct='%1.1f%%', label='', colors=[columns_labels[v] for v in pxxx5.keys()])


axx3 = plt.subplot(1, 5, 5);
pxxx3 = graduacao_imd_2020_01[graduacao_imd_2020_01.duplicated(subset = ["discente", "semestre"], keep = 'first') != True]
pxxx3 = pxxx3[pxxx3["descricao"] != "INDEFERIDO"]
pxxx3 = pxxx3[pxxx3["descricao"] != "EXCLUIDA"]
pxxx3 = pxxx3[pxxx3["descricao"] != "CANCELADO"]

plt.title('2020.1');
pxxx3 = pxxx3.descricao.value_counts()
pxxx3.plot(kind='pie', autopct='%1.1f%%', label='', colors=[columns_labels[v] for v in pxxx3.keys()])

print('Matrículas')

In [ ]:
graduacao_imd_2018_01 = graduacao_imd_2018_01.loc[graduacao_imd_2018_01['disciplina'] == 'INTRODUÇÃO ÀS TÉCNICAS DE PROGRAMAÇÃO']
graduacao_imd_2018_01 = graduacao_imd_2018_01.loc[graduacao_imd_2018_01["descricao"] != "INDEFERIDO"]
graduacao_imd_2018_01 = graduacao_imd_2018_01.loc[graduacao_imd_2018_01["descricao"] != "DESISTENCIA"]
graduacao_imd_2018_01 = graduacao_imd_2018_01.loc[graduacao_imd_2018_01["descricao"] != "EXCLUIDA"]
graduacao_imd_2018_01 = graduacao_imd_2018_01.loc[graduacao_imd_2018_01["descricao"] != "CANCELADO"]
graduacao_imd_2018_01 = graduacao_imd_2018_01.loc[graduacao_imd_2018_01["descricao"] != "TRANCADO"]
rotulos_1 = graduacao_imd_2018_01.apply(matematicamentereprovado, axis = 1)
graduacao_imd_2018_01["reprovadomatematicamente"] = rotulos_1
graduacao_imd_2018_01 = graduacao_imd_2018_01.loc[graduacao_imd_2018_01['reprovadomatematicamente'] == False]


graduacao_imd_2018_02 = graduacao_imd_2018_02.loc[graduacao_imd_2018_02['disciplina'] == 'INTRODUÇÃO ÀS TÉCNICAS DE PROGRAMAÇÃO']
graduacao_imd_2018_02 = graduacao_imd_2018_02.loc[graduacao_imd_2018_02["descricao"] != "INDEFERIDO"]
graduacao_imd_2018_02 = graduacao_imd_2018_02.loc[graduacao_imd_2018_02["descricao"] != "DESISTENCIA"]
graduacao_imd_2018_02 = graduacao_imd_2018_02.loc[graduacao_imd_2018_02["descricao"] != "EXCLUIDA"]
graduacao_imd_2018_02 = graduacao_imd_2018_02.loc[graduacao_imd_2018_02["descricao"] != "CANCELADO"]
graduacao_imd_2018_02 = graduacao_imd_2018_02.loc[graduacao_imd_2018_02["descricao"] != "TRANCADO"]
rotulos_2 = graduacao_imd_2018_02.apply(matematicamentereprovado, axis = 1)
graduacao_imd_2018_02["reprovadomatematicamente"] = rotulos_2
graduacao_imd_2018_02 = graduacao_imd_2018_02.loc[graduacao_imd_2018_02['reprovadomatematicamente'] == False]


graduacao_imd_2019_01 = graduacao_imd_2019_01.loc[graduacao_imd_2019_01['disciplina'] == 'INTRODUÇÃO ÀS TÉCNICAS DE PROGRAMAÇÃO']
graduacao_imd_2019_01 = graduacao_imd_2019_01.loc[graduacao_imd_2019_01["descricao"] != "INDEFERIDO"]
graduacao_imd_2019_01 = graduacao_imd_2019_01.loc[graduacao_imd_2019_01["descricao"] != "DESISTENCIA"]
graduacao_imd_2019_01 = graduacao_imd_2019_01.loc[graduacao_imd_2019_01["descricao"] != "EXCLUIDA"]
graduacao_imd_2019_01 = graduacao_imd_2019_01.loc[graduacao_imd_2019_01["descricao"] != "CANCELADO"]
graduacao_imd_2019_01 = graduacao_imd_2019_01.loc[graduacao_imd_2019_01["descricao"] != "TRANCADO"]
rotulos_3 = graduacao_imd_2019_01.apply(matematicamentereprovado, axis = 1)
graduacao_imd_2019_01["reprovadomatematicamente"] = rotulos_3
graduacao_imd_2019_01 = graduacao_imd_2019_01.loc[graduacao_imd_2019_01['reprovadomatematicamente'] == False]

graduacao_imd_2019_02 = graduacao_imd_2019_02.loc[graduacao_imd_2019_02['disciplina'] == 'INTRODUÇÃO ÀS TÉCNICAS DE PROGRAMAÇÃO']
graduacao_imd_2019_02 = graduacao_imd_2019_02.loc[graduacao_imd_2019_02["descricao"] != "INDEFERIDO"]
graduacao_imd_2019_02 = graduacao_imd_2019_02.loc[graduacao_imd_2019_02["descricao"] != "DESISTENCIA"]
graduacao_imd_2019_02 = graduacao_imd_2019_02.loc[graduacao_imd_2019_02["descricao"] != "EXCLUIDA"]
graduacao_imd_2019_02 = graduacao_imd_2019_02.loc[graduacao_imd_2019_02["descricao"] != "CANCELADO"]
graduacao_imd_2019_02 = graduacao_imd_2019_02.loc[graduacao_imd_2019_02["descricao"] != "TRANCADO"]
rotulos_4 = graduacao_imd_2019_02.apply(matematicamentereprovado, axis = 1)
graduacao_imd_2019_02["reprovadomatematicamente"] = rotulos_4
graduacao_imd_2019_02 = graduacao_imd_2019_02.loc[graduacao_imd_2019_02['reprovadomatematicamente'] == False]

graduacao_imd_2020_01 = graduacao_imd_2020_01.loc[graduacao_imd_2020_01['disciplina'] == 'INTRODUÇÃO ÀS TÉCNICAS DE PROGRAMAÇÃO']
graduacao_imd_2020_01 = graduacao_imd_2020_01.loc[graduacao_imd_2020_01["descricao"] != "INDEFERIDO"]
graduacao_imd_2020_01 = graduacao_imd_2020_01.loc[graduacao_imd_2020_01["descricao"] != "DESISTENCIA"]
graduacao_imd_2020_01 = graduacao_imd_2020_01.loc[graduacao_imd_2020_01["descricao"] != "EXCLUIDA"]
graduacao_imd_2020_01 = graduacao_imd_2020_01.loc[graduacao_imd_2020_01["descricao"] != "CANCELADO"]
graduacao_imd_2020_01 = graduacao_imd_2020_01.loc[graduacao_imd_2020_01["descricao"] != "TRANCADO"]
rotulos_5 = graduacao_imd_2020_01.apply(matematicamentereprovado, axis = 1)
graduacao_imd_2020_01["reprovadomatematicamente"] = rotulos_5
graduacao_imd_2020_01 = graduacao_imd_2020_01.loc[graduacao_imd_2020_01['reprovadomatematicamente'] == False]

graduacao_imd_2018_02.descricao.unique()

In [ ]:
#Realizando os cálculos da taxa de aprovação de cada um dos semestres
itp_2018_01 = graduacao_imd_2018_01[graduacao_imd_2018_01.duplicated(subset = ["discente", "semestre"], keep = 'first') != True]
aprovados_itp_2018_01 = itp_2018_01.pivot_table(index = ["disciplina"], values = "aprovado", aggfunc = np.sum)
quantidade_alunos = itp_2018_01.pivot_table(index = "disciplina", values = "capacidade_aluno", aggfunc = len)
percent = (aprovados_itp_2018_01.aprovado/quantidade_alunos.capacidade_aluno)*100
aprovados_itp_2018_01.aprovado = percent

itp_2018_02 = graduacao_imd_2018_02[graduacao_imd_2018_02.duplicated(subset = ["discente", "semestre"], keep = 'first') != True]
aprovados_itp_2018_02 = itp_2018_02.pivot_table(index = ["disciplina"], values = "aprovado", aggfunc = np.sum)
quantidade_alunos = itp_2018_02.pivot_table(index = "disciplina", values = "capacidade_aluno", aggfunc = len)
percent = (aprovados_itp_2018_02.aprovado/quantidade_alunos.capacidade_aluno)*100
aprovados_itp_2018_02.aprovado = percent

itp_2019_01 = graduacao_imd_2019_01[graduacao_imd_2019_01.duplicated(subset = ["discente", "semestre"], keep = 'first') != True]
aprovados_itp_2019_01 = itp_2019_01.pivot_table(index = ["disciplina"], values = 'aprovado' , aggfunc = np.sum)
quantidade_alunos = itp_2019_01.pivot_table(index = "disciplina", values = "capacidade_aluno", aggfunc = len)
percent = (aprovados_itp_2019_01.aprovado/quantidade_alunos.capacidade_aluno)*100
aprovados_itp_2019_01.aprovado = percent

itp_2019_02 = graduacao_imd_2019_02[graduacao_imd_2019_02.duplicated(subset = ["discente", "semestre"], keep = 'first') != True]
aprovados_itp_2019_02 = itp_2019_02.pivot_table(index = ["disciplina"], values = 'aprovado' , aggfunc = np.sum)
quantidade_alunos = itp_2019_02.pivot_table(index = "disciplina", values = "capacidade_aluno", aggfunc = len)
percent = (aprovados_itp_2019_02.aprovado/quantidade_alunos.capacidade_aluno)*100
aprovados_itp_2019_02.aprovado = percent

itp_2020_01 = graduacao_imd_2020_01[graduacao_imd_2020_01.duplicated(subset = ["discente", "semestre"], keep = 'first') != True]
aprovados_itp_2020_01 = itp_2020_01.pivot_table(index = ["disciplina"], values = 'aprovado' , aggfunc = np.sum)
quantidade_alunos = itp_2020_01.pivot_table(index = "disciplina", values = "capacidade_aluno", aggfunc = len)
percent = (aprovados_itp_2020_01.aprovado/quantidade_alunos.capacidade_aluno)*100
aprovados_itp_2020_01.aprovado = percent


In [ ]:
#Realizando os cálculos da taxa de aprovação por horário
itp_2018_01 = graduacao_imd_2018_01[graduacao_imd_2018_01.duplicated(subset = ["discente", "semestre"], keep = 'first') != True]
aprovados_itp_2018_01_horario = itp_2018_01.pivot_table(index = ["descricao_horario"], values = "aprovado", aggfunc = np.sum)
quantidade_alunos = itp_2018_01.pivot_table(index = "descricao_horario", values = "capacidade_aluno", aggfunc = len)
percent = (aprovados_itp_2018_01_horario.aprovado/quantidade_alunos.capacidade_aluno)*100
aprovados_itp_2018_01_horario.aprovado = percent

itp_2018_02 = graduacao_imd_2018_02[graduacao_imd_2018_02.duplicated(subset = ["discente", "semestre"], keep = 'first') != True]
aprovados_itp_2018_02_horario = itp_2018_02.pivot_table(index = ["descricao_horario"], values = "aprovado", aggfunc = np.sum)
quantidade_alunos = itp_2018_02.pivot_table(index = "descricao_horario", values = "capacidade_aluno", aggfunc = len)
percent = (aprovados_itp_2018_02_horario.aprovado/quantidade_alunos.capacidade_aluno)*100
aprovados_itp_2018_02_horario.aprovado = percent

itp_2019_01 = graduacao_imd_2019_01[graduacao_imd_2019_01.duplicated(subset = ["discente", "semestre"], keep = 'first') != True]
aprovados_itp_2019_01_horario = itp_2019_01.pivot_table(index = ["descricao_horario"], values = 'aprovado' , aggfunc = np.sum)
quantidade_alunos = itp_2019_01.pivot_table(index = "descricao_horario", values = "capacidade_aluno", aggfunc = len)
percent = (aprovados_itp_2019_01_horario.aprovado/quantidade_alunos.capacidade_aluno)*100
aprovados_itp_2019_01_horario.aprovado = percent

itp_2019_02 = graduacao_imd_2019_02[graduacao_imd_2019_02.duplicated(subset = ["discente", "semestre"], keep = 'first') != True]
aprovados_itp_2019_02_horario = itp_2019_02.pivot_table(index = ["descricao_horario"], values = 'aprovado' , aggfunc = np.sum)
quantidade_alunos = itp_2019_02.pivot_table(index = "descricao_horario", values = "capacidade_aluno", aggfunc = len)
percent = (aprovados_itp_2019_02_horario.aprovado/quantidade_alunos.capacidade_aluno)*100
aprovados_itp_2019_02_horario.aprovado = percent

itp_2020_01 = graduacao_imd_2020_01[graduacao_imd_2020_01.duplicated(subset = ["discente", "semestre"], keep = 'first') != True]
aprovados_itp_2020_01_horario = itp_2020_01.pivot_table(index = ["descricao_horario"], values = 'aprovado' , aggfunc = np.sum)
quantidade_alunos = itp_2020_01.pivot_table(index = "descricao_horario", values = "capacidade_aluno", aggfunc = len)
percent = (aprovados_itp_2020_01_horario.aprovado/quantidade_alunos.capacidade_aluno)*100
aprovados_itp_2020_01_horario.aprovado = percent

In [ ]:
medias_por_semestre = []

p1 = graduacao_imd_2018_01[graduacao_imd_2018_01.duplicated(subset = ["discente", "semestre"], keep = 'first') != True]
p1 = pd.DataFrame(p1,columns = ['id_turma', 'discente','media_final'])
p1 = p1[p1['media_final'].notna()]
sum = 0;
for nota in p1.media_final:
  sum += float(nota);
mean = sum / p1.media_final.size
medias_por_semestre.append(mean)


p2 = graduacao_imd_2018_02[graduacao_imd_2018_02.duplicated(subset = ["discente", "semestre"], keep = 'first') != True]
p2 = pd.DataFrame(p2,columns = ['id_turma', 'discente','media_final'])
p2 = p2[p2['media_final'].notna()]
sum = 0;
for nota in p2.media_final:
  sum += float(nota);
mean = sum / p2.media_final.size
medias_por_semestre.append(mean)


p6 = graduacao_imd_2019_02[graduacao_imd_2019_02.duplicated(subset = ["discente", "semestre"], keep = 'first') != True]
#itp_2018_02 = itp_2018_02.pivot_table(index = ["id_turma"])
p6 = pd.DataFrame(p6,columns = ['id_turma', 'discente','media_final'])
p6 = p6[p6['media_final'].notna()]
sum = 0;
for nota in p6.media_final:
  sum += float(nota);
mean = sum / p6.media_final.size
medias_por_semestre.append(mean)


p3 = graduacao_imd_2019_02[graduacao_imd_2019_02.duplicated(subset = ["discente", "semestre"], keep = 'first') != True]
#itp_2018_02 = itp_2018_02.pivot_table(index = ["id_turma"])
p3 = pd.DataFrame(p3,columns = ['id_turma', 'discente','media_final'])
p3 = p3[p3['media_final'].notna()]
sum = 0;
for nota in p3.media_final:
  sum += float(nota);
mean = sum / p3.media_final.size
medias_por_semestre.append(mean)


p5 = graduacao_imd_2020_01[graduacao_imd_2020_01.duplicated(subset = ["discente", "semestre"], keep = 'first') != True]
p5 = pd.DataFrame(p5,columns = ['id_turma', 'discente','media_final'])
p5 = p5[p5['media_final'].notna()]
sum = 0;
for nota in p5.media_final:
  sum += float(nota);
mean = sum / p5.media_final.size
medias_por_semestre.append(mean)


p4 = pd.DataFrame(graduacao_imd_2018_02,columns = ['id_turma', 'discente','media_final'])
#graduacao_imd_2018_02

In [ ]:
aprovados_itp_2019_02_horario

In [ ]:
fig = plt.figure(figsize=(15, 5));

frames = [graduacao_imd_2018_01, graduacao_imd_2018_02, graduacao_imd_2019_01, graduacao_imd_2019_02, graduacao_imd_2020_01] 
result = pd.concat(frames)
plt.scatter(result.media_final, result.numero_total_faltas)

In [ ]:
%matplotlib inline 

def reduce(str, n):
  str_splited = str.split();
  str = '';
  for i in range(n):
    str += str_splited[i];
    if i+1 != n:
      str += ' ';
  return str;

def reduce_names(arr, n):
  n_arr = [None] * len(arr);
  for i in range(len(arr)):
    n_arr[i] = reduce(arr[i], n);
  return n_arr;

print('Aprovação/Reprovação por semestre')

fig = plt.figure(figsize=(20, 6));


plt.subplot(1, 5, 1);
plt.title('2018.1');
itp_2018_01.aprovado.value_counts().plot(kind='pie', labels=['Aprovado', 'Reprovado'], autopct='%.2f%%');
plt.ylabel('');

plt.subplot(1, 5, 2);
plt.title('2018.2');
itp_2018_02.aprovado.value_counts().plot(kind='pie', labels=['Aprovado', 'Reprovado'], autopct='%.2f%%');
plt.ylabel('');

plt.subplot(1, 5, 3);
plt.title('2019.1');
itp_2019_01.aprovado.value_counts().plot(kind='pie', labels=['Aprovado', 'Reprovado'], autopct='%.2f%%');
plt.ylabel('');

plt.subplot(1, 5, 4);
plt.title('2019.2');
itp_2019_02.aprovado.value_counts().plot(kind='pie', labels=['Aprovado', 'Reprovado'], autopct='%.2f%%');
plt.ylabel('');

plt.subplot(1, 5, 5);
plt.title('2020.1');
itp_2020_01.aprovado.value_counts().plot(kind='pie', labels=['Aprovado', 'Reprovado'], autopct='%.2f%%');
plt.ylabel('');

plt.show();

print('Aprovação por turma');

fig = plt.figure(figsize=(20, 6));

plt.subplot(1, 5, 1);
plt.title('2018.1');
aprovados_itp_2018_01_horario.aprovado.plot(kind='bar', width=0.5);
plt.xticks(range(len(aprovados_itp_2018_01_horario.index.values)), reduce_names(aprovados_itp_2018_01_horario.index.values, 1), rotation='horizontal');
plt.xlabel('');
plt.ylim(0, 100);


plt.subplot(1, 5, 2);
plt.title('2018.2');
aprovados_itp_2018_02_horario.aprovado.plot(kind='bar', width=0.5);
plt.xticks(range(len(aprovados_itp_2018_02_horario.index.values)), reduce_names(aprovados_itp_2018_02_horario.index.values, 1), rotation='horizontal');
plt.xlabel('');
plt.ylim(0, 100);

plt.subplot(1, 5, 3);
plt.title('2019.1');
aprovados_itp_2019_01_horario.aprovado.plot(kind='bar', width=0.5);
plt.xticks(range(len(aprovados_itp_2019_01_horario.index.values)), reduce_names(aprovados_itp_2019_01_horario.index.values, 1), rotation='horizontal');
plt.xlabel('');
plt.ylim(0, 100);

plt.subplot(1, 5, 4);
plt.title('2019.2');
aprovados_itp_2019_02_horario.aprovado.plot(kind='bar', width=0.5);
plt.xticks(range(len(aprovados_itp_2019_02_horario.index.values)), reduce_names(aprovados_itp_2019_02_horario.index.values, 1), rotation='horizontal');
plt.xlabel('');
plt.ylim(0, 100);

plt.subplot(1, 5, 5);
plt.title('2020.1');
aprovados_itp_2020_01_horario.aprovado.plot(kind='bar', width=0.5);
plt.xticks(range(len(aprovados_itp_2020_01_horario.index.values)), reduce_names(aprovados_itp_2020_01_horario.index.values, 1), rotation='horizontal');
plt.xlabel('');
plt.ylim(0, 100);

plt.show();
  
print('Média por semestre');

fig = plt.figure(figsize=(4, 4));
medias_por_semestre_frame = pd.DataFrame(medias_por_semestre);
medias_por_semestre_frame.plot(kind='bar', width=0.5)
plt.title('Média final dos alunos')
plt.xticks(range((len(medias_por_semestre))), ['2018.1', '2018.2', '2019.1', '2019.2', '2020.1'], rotation='horizontal');
#plt.xlabel('');
plt.legend().remove()
plt.ylim(0, 10);

plt.show();

pxx1 = graduacao_imd_2018_01[graduacao_imd_2018_01.duplicated(subset = ["discente", "semestre"], keep = 'first') != True]
pxx1 = pxx1.replace({',': '.'}, regex=True)
pxx1["media_final"] = pd.to_numeric(pxx1["media_final"])
pxx1.boxplot(column='media_final')

pxx4 = graduacao_imd_2018_02[graduacao_imd_2018_02.duplicated(subset = ["discente", "semestre"], keep = 'first') != True]
pxx4 = pxx4.replace({',': '.'}, regex=True)
pxx4["media_final"] = pd.to_numeric(pxx4["media_final"])
pxx4.boxplot(column='media_final')

pxx5 = graduacao_imd_2019_01[graduacao_imd_2019_01.duplicated(subset = ["discente", "semestre"], keep = 'first') != True]
pxx5 = pxx5.replace({',': '.'}, regex=True)
pxx5["media_final"] = pd.to_numeric(pxx5["media_final"])
pxx5.boxplot(column='media_final')

pxx2 = graduacao_imd_2019_02[graduacao_imd_2019_02.duplicated(subset = ["discente", "semestre"], keep = 'first') != True]
pxx2 = pxx2.replace({',': '.'}, regex=True)
pxx2["media_final"] = pd.to_numeric(pxx2["media_final"])
pxx2.boxplot(column='media_final')

pxx3 = graduacao_imd_2020_01[graduacao_imd_2020_01.duplicated(subset = ["discente", "semestre"], keep = 'first') != True]
pxx3 = pxx3.replace({',': '.'}, regex=True)
pxx3["media_final"] = pd.to_numeric(pxx3["media_final"])


print('Média final por semestre');

fig = plt.figure(figsize=(20, 6));

ax1 = plt.subplot(1, 5, 1);
fig.add_subplot(ax1)
pxx1.boxplot(column='media_final')
plt.title('2018.1');
#plt.xticks(range(len(pxx1.index.values)), reduce_names(aprovados_itp_2018_02_horario.index.values, 1), rotation='horizontal');
plt.xlabel('');
plt.ylim(0, 10);

ax4 = plt.subplot(1, 5, 2);
fig.add_subplot(ax4)
pxx4.boxplot(column='media_final')
plt.title('2018.2');
#plt.xticks(range(len(pxx1.index.values)), reduce_names(aprovados_itp_2018_02_horario.index.values, 1), rotation='horizontal');
plt.xlabel('');
plt.ylim(0, 10);

ax2 = plt.subplot(1, 5, 3);
fig.add_subplot(ax2)
pxx2.boxplot(column='media_final')
plt.title('2019.1');
#plt.xticks(range(len(aprovados_itp_2019_02_horario.index.values)), reduce_names(aprovados_itp_2019_02_horario.index.values, 1), rotation='horizontal');
plt.xlabel('');
plt.ylim(0, 10);

ax5 = plt.subplot(1, 5, 4);
fig.add_subplot(ax5)
pxx5.boxplot(column='media_final')
plt.title('2019.2');
#plt.xticks(range(len(aprovados_itp_2019_02_horario.index.values)), reduce_names(aprovados_itp_2019_02_horario.index.values, 1), rotation='horizontal');
plt.xlabel('');
plt.ylim(0, 10);

ax3 = plt.subplot(1, 5, 5);
fig.add_subplot(ax3)
pxx3.boxplot(column='media_final')
plt.title('2020.1');
#plt.xticks(range(len(aprovados_itp_2020_01_horario.index.values)), reduce_names(aprovados_itp_2020_01_horario.index.values, 1), rotation='horizontal');
plt.xlabel('');
plt.ylim(0, 10);

plt.show();

print('Distribuição das notas')
fig = plt.figure(figsize=(20, 6));


plt.subplot(1, 5, 1);
plt.title('2018.1');

px4 = graduacao_imd_2018_01.replace({',': '.'}, regex=True)
px4["nota"] = pd.to_numeric(px4["nota"])
px4["media_final"] = pd.to_numeric(px4["media_final"])
plt.hist(px4.dropna().nota, bins='auto', density=True)
plt.ylim(0, 0.33);


plt.subplot(1, 5, 2);
plt.title('2018.2');

px1 = graduacao_imd_2018_02.replace({',': '.'}, regex=True)
px1["nota"] = pd.to_numeric(px1["nota"])
px1["media_final"] = pd.to_numeric(px1["media_final"])
plt.hist(px1.dropna().nota, bins='auto', density=True)
plt.ylim(0, 0.33);


plt.subplot(1, 5, 3);

px5 = graduacao_imd_2019_01.replace({',': '.'}, regex=True)
px5["nota"] = pd.to_numeric(px5["nota"])
px5["media_final"] = pd.to_numeric(px5["media_final"])
plt.title('2019.1')
plt.hist(px5.dropna().nota, bins='auto', density=True)
plt.ylim(0, 0.33);


plt.subplot(1, 5, 4);

px2 = graduacao_imd_2019_02.replace({',': '.'}, regex=True)
px2["nota"] = pd.to_numeric(px2["nota"])
px2["media_final"] = pd.to_numeric(px2["media_final"])
plt.title('2019.2')
plt.hist(px2.dropna().nota, bins='auto', density=True)
plt.ylim(0, 0.33);

plt.subplot(1, 5, 5);
plt.title('2020.1');

px3 = graduacao_imd_2020_01.replace({',': '.'}, regex=True)
px3["nota"] = pd.to_numeric(px3["nota"])
px3["media_final"] = pd.to_numeric(px3["media_final"])
plt.title('2020.1')
plt.hist(px3.dropna().nota, bins='auto', density=True)
plt.ylim(0, 0.33);

plt.show()




In [ ]:
# A PARTIR DAQUI REFAZER OS K-MEANS

In [ ]:
kmeansdb = base_final

dfkm = kmeansdb[['discente', 'docente', 'unidade', 'nota', 'media_final','numero_total_faltas', 'aprovado', 'reposicao', 'reprovadoporfalta']]


def strhash_to_inthash(row):
  strhash = row['discente'];
  return hash(strhash) % 10**8;

def bool_to_int_aprovado(row):
  if row['aprovado'] == True:
    return 1;
  else:
    return 0;

def bool_to_int_reprovadoporfalta(row):
  if row['reprovadoporfalta'] == True:
    return 1;
  else:
    return 0;

def bool_to_int_reposicao(row):
  if row['reposicao'] == True:
    return 1;
  else:
    return 0;

def docentehash(row):
  strhash = row['docente'].split()[0];
  inthash = hash(strhash) % 10**8;
  return inthash;

inthashed_docente = dfkm.apply(docentehash, axis = 1)
dfkm["docente"] = inthashed_docente

inthashed_discente = dfkm.apply(strhash_to_inthash, axis = 1)
dfkm["discente"] = inthashed_discente 

aprovacao = dfkm.apply(bool_to_int_aprovado, axis = 1)
dfkm["aprovado"] = aprovacao 

reprovacaoporfalta = dfkm.apply(bool_to_int_reprovadoporfalta, axis = 1)
dfkm["reprovadoporfalta"] = reprovacaoporfalta 

reposicao = dfkm.apply(bool_to_int_reposicao, axis = 1)
dfkm["reposicao"] = reposicao 

#'''
normalized_docente=(dfkm.docente-dfkm.docente.min())/(dfkm.docente.max()-dfkm.docente.min())
dfkm["docente"] = normalized_docente
#'''

#'''
normalized_discente=(dfkm.discente-dfkm.discente.min())/(dfkm.discente.max()-dfkm.discente.min())
dfkm["discente"] = normalized_discente
#'''

#'''
normalized_unidade=(dfkm.unidade-dfkm.unidade.min())/(dfkm.unidade.max()-dfkm.unidade.min())
dfkm["unidade"] = normalized_unidade
#'''


normalized_nota=(dfkm.nota-dfkm.nota.min())/(dfkm.nota.max()-dfkm.nota.min())
dfkm["nota"] = normalized_nota

normalized_media_final=(dfkm.media_final-dfkm.media_final.min())/(dfkm.media_final.max()-dfkm.media_final.min())
dfkm["media_final"] = normalized_media_final

normalized_numero_total_faltas=(dfkm.numero_total_faltas-dfkm.numero_total_faltas.min())/(dfkm.numero_total_faltas.max()-dfkm.numero_total_faltas.min())
dfkm["numero_total_faltas"] = normalized_numero_total_faltas

dfkm
#dfkm[dfkm["reprovadoporfalta"] == 1]
#dfkm[dfkm["aprovado"] == 1]
#dfkm[dfkm["media_final"] == 1]
#dfkm[dfkm["nota"] == 1]

# TODAS CARACTERISTICAS COM VALORES ENTE [0, 1] (inclusive)
# DISCENTE/DOCENTE COM HASH EM FLOAT


In [ ]:
#dfkm.info()

dfkm.aprovado = dfkm.aprovado.astype(np.float64)
dfkm.reprovadoporfalta = dfkm.reprovadoporfalta.astype(np.float64)
dfkm.reposicao = dfkm.reposicao.astype(np.float64)

dfkm.info()

In [ ]:
dfkm

In [ ]:
describedb = dfkm
describedb = dfkm[['nota', 'media_final', 'numero_total_faltas', 'aprovado']]

describedb.describe()

In [ ]:

# Selecionando o número de clusters através do método Elbow (Soma das distâncias quadráticas intra clusters):
X2 = base_final[['unidade','nota']].iloc[: , :].values
inertia = []
for n in range(1 , 11):
    algorithm = (KMeans(n_clusters = n))
    algorithm.fit(X2)
    inertia.append(algorithm.inertia_)

plt.figure(1 , figsize = (15 ,6))
plt.plot(np.arange(1 , 11) , inertia , 'o')
plt.plot(np.arange(1 , 11) , inertia , '-' , alpha = 0.5)
plt.xlabel('Número de Clusters') , plt.ylabel('Soma das Distâncias Q intra Clusters')
plt.show()

# Inicializando e Computando o KMeans com o valor de 4 clusters:
algorithm = (KMeans(n_clusters = 3))
algorithm.fit(X2)

# Saída:

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=4, n_init=10,
       random_state=None, tol=0.0001, verbose=0)

# Visualizando os grupos criados e seus centroides:
labels2 = algorithm.labels_
centroids2 = algorithm.cluster_centers_

h = 0.02
x_min, x_max = X2[:, 0].min() - 1, X2[:, 0].max() + 1
y_min, y_max = X2[:, 1].min() - 1, X2[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
Z = algorithm.predict(np.c_[xx.ravel(), yy.ravel()]) 

plt.figure(1 , figsize = (15 , 7) )
plt.clf()
Z2 = Z.reshape(xx.shape)
plt.imshow(Z2 , interpolation='nearest', extent=(xx.min(), xx.max(), yy.min(), yy.max()), cmap = plt.cm.Pastel2, aspect = 'auto', origin='lower')
plt.scatter( x = 'unidade' ,y = 'nota' , data = base_final , c = labels2 , s = 200 )
plt.scatter(x = centroids2[: , 0] , y =  centroids2[: , 1] , s = 300 , c = 'red' , alpha = 0.5)
plt.ylabel('Nota') , plt.xlabel('Unidade')
plt.show()

In [ ]:
X2 = base_final[['nota','media_final']].iloc[: , :].values
inertia = []
for n in range(1 , 11):
    algorithm = (KMeans(n_clusters = n))
    algorithm.fit(X2)
    inertia.append(algorithm.inertia_)

plt.figure(1 , figsize = (15 ,6))
plt.plot(np.arange(1 , 11) , inertia , 'o')
plt.plot(np.arange(1 , 11) , inertia , '-' , alpha = 0.5)
plt.xlabel('Número de Clusters') , plt.ylabel('Soma das Distâncias Q intra Clusters')
plt.show()

# Inicializando e Computando o KMeans com o valor de 4 clusters:
algorithm = (KMeans(n_clusters = 3))
algorithm.fit(X2)

# Saída:

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=4, n_init=10,
       random_state=None, tol=0.0001, verbose=0)

# Visualizando os grupos criados e seus centroides:
labels2 = algorithm.labels_
centroids2 = algorithm.cluster_centers_

h = 0.02
x_min, x_max = X2[:, 0].min() - 1, X2[:, 0].max() + 1
y_min, y_max = X2[:, 1].min() - 1, X2[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
Z = algorithm.predict(np.c_[xx.ravel(), yy.ravel()]) 

plt.figure(1 , figsize = (15 , 7) )
plt.clf()
Z2 = Z.reshape(xx.shape)
plt.imshow(Z2 , interpolation='nearest', extent=(xx.min(), xx.max(), yy.min(), yy.max()), cmap = plt.cm.Pastel2, aspect = 'auto', origin='lower')
plt.scatter( x = 'nota' ,y = 'media_final' , data = base_final , c = labels2 , s = 200 )
plt.scatter(x = centroids2[: , 0] , y =  centroids2[: , 1] , s = 300 , c = 'red' , alpha = 0.5)
plt.ylabel('Média Final') , plt.xlabel('Nota')
plt.show()

In [ ]:
X2 = base_final[['docente','media_final']].iloc[: , :].values
inertia = []
for n in range(1 , 11):
    algorithm = (KMeans(n_clusters = n))
    algorithm.fit(X2)
    inertia.append(algorithm.inertia_)

plt.figure(1 , figsize = (15 ,6))
plt.plot(np.arange(1 , 11) , inertia , 'o')
plt.plot(np.arange(1 , 11) , inertia , '-' , alpha = 0.5)
plt.xlabel('Número de Clusters') , plt.ylabel('Soma das Distâncias Q intra Clusters')
plt.show()

# Inicializando e Computando o KMeans com o valor de 4 clusters:
algorithm = (KMeans(n_clusters = 3))
algorithm.fit(X2)

# Saída:

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=4, n_init=10,
       random_state=None, tol=0.0001, verbose=0)

# Visualizando os grupos criados e seus centroides:
labels2 = algorithm.labels_
centroids2 = algorithm.cluster_centers_

h = 0.02
x_min, x_max = X2[:, 0].min() - 1, X2[:, 0].max() + 1
y_min, y_max = X2[:, 1].min() - 1, X2[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
Z = algorithm.predict(np.c_[xx.ravel(), yy.ravel()]) 

plt.figure(1 , figsize = (15 , 7) )
plt.clf()
Z2 = Z.reshape(xx.shape)
plt.imshow(Z2 , interpolation='nearest', extent=(xx.min(), xx.max(), yy.min(), yy.max()), cmap = plt.cm.Pastel2, aspect = 'auto', origin='lower')
plt.scatter( x = 'docente' ,y = 'media_final' , data = base_final , c = labels2 , s = 200 )
plt.scatter(x = centroids2[: , 0] , y =  centroids2[: , 1] , s = 300 , c = 'red' , alpha = 0.5)
plt.ylabel('Média Final') , plt.xlabel('Docente')
plt.show()

In [ ]:

# Inicializando e Computando o KMeans com o valor de 4 clusters:
algorithm = (KMeans(n_clusters = 4))
algorithm.fit(X2)

# Saída:

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=4, n_init=10,
       random_state=None, tol=0.0001, verbose=0)

# Visualizando os grupos criados e seus centroides:
labels2 = algorithm.labels_
centroids2 = algorithm.cluster_centers_

h = 0.02
x_min, x_max = X2[:, 0].min() - 1, X2[:, 0].max() + 1
y_min, y_max = X2[:, 1].min() - 1, X2[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
Z = algorithm.predict(np.c_[xx.ravel(), yy.ravel()]) 

plt.figure(1 , figsize = (22 , 7) )
plt.clf()
Z2 = Z.reshape(xx.shape)
plt.imshow(Z2 , interpolation='nearest', extent=(xx.min(), xx.max(), yy.min(), yy.max()), cmap = plt.cm.Pastel2, aspect = 'auto', origin='lower')
plt.scatter( x = 'docente' ,y = 'aprovado' , data = base_final , c = labels2 , s = 200 )
plt.scatter(x = centroids2[: , 0] , y =  centroids2[: , 1] , s = 300 , c = 'red' , alpha = 0.5)
plt.ylabel('Aprovado') , plt.xlabel('Docente')
plt.show()

In [ ]:

# Inicializando e Computando o KMeans com o valor de 4 clusters:
algorithm = (KMeans(n_clusters = 4))
algorithm.fit(X2)

# Saída:

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=4, n_init=10,
       random_state=None, tol=0.0001, verbose=0)

# Visualizando os grupos criados e seus centroides:
labels2 = algorithm.labels_
centroids2 = algorithm.cluster_centers_

h = 0.02
x_min, x_max = X2[:, 0].min() - 1, X2[:, 0].max() + 1
y_min, y_max = X2[:, 1].min() - 1, X2[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
Z = algorithm.predict(np.c_[xx.ravel(), yy.ravel()]) 

plt.figure(1 , figsize = (22 , 7) )
plt.clf()
Z2 = Z.reshape(xx.shape)
plt.imshow(Z2 , interpolation='nearest', extent=(xx.min(), xx.max(), yy.min(), yy.max()), cmap = plt.cm.Pastel2, aspect = 'auto', origin='lower')
plt.scatter( x = 'docente' ,y = 'descricao' , data = base_final , c = labels2 , s = 200 )
plt.scatter(x = centroids2[: , 0] , y =  centroids2[: , 1] , s = 300 , c = 'red' , alpha = 0.5)
plt.ylabel('Matrícula') , plt.xlabel('Professor')
plt.show()